<a href="https://colab.research.google.com/github/DigitalHugManitees/DH_Topic_Workshop/blob/main/OCR_and_LDA_superbook_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OCR and LDA for pdf files.
This super notebook is for a concise analysis of the dalhousie gazette pdf files. 




In [1]:

"""
1. sign into your google account
2. save a copy of this notebook to your google drive as your own
3. run cell linking to your google drive

"""

#mount google drive here
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 

# Set your working directory to a folder in your Google Drive. 
# the base Google Drive directory
root_dir = "/content/drive/My Drive/"
# choose where you want your project files to be saved
project_folder = "Colab Notebooks/OCR_Project_Folder/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # show me the current working directory

  print('\nYour working directory was changed to ' + root_dir + project_folder)

create_and_set_working_directory(project_folder)

print()
print('Move your pdf files to the working directory: ' + project_folder)

Mounted at /content/drive

Your working directory was changed to /content/drive/My Drive/Colab Notebooks/OCR_Project_Folder/

Move your pdf files to the working directory: Colab Notebooks/OCR_Project_Folder/


In [2]:
"""
Install dependencies and libraries
"""

!apt install tesseract-ocr
!apt install libtesseract-dev
!apt-get install poppler-utils
!pip install pdf2image
!pip install Pillow
!pip install pytesseract

# Import libraries
import pytesseract
from PIL import ImageEnhance, ImageFilter, Image
import sys
from pdf2image import convert_from_path
import os
from google.colab import files
from tqdm import tqdm

#sometimes poppler install fails - this line should update everything
!apt-get update 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.14).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use

In [ ]:
"""
Optional: Change runtime type from CPU to GPU.
"""
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13926517785464551111
 xla_global_id: -1]

In [3]:
"""
Step 3: gather the PDF files and create the OsCaRizer function

This should find all PDF files in working dir. 
HOWEVER, it will not find those nested in subdirectories
"""
path_of_the_directory = os.getcwd()
ext = ('.pdf')
pdf_list = []
for files in os.listdir(path_of_the_directory):
    if files.endswith(ext):
        print(files)
        pdf_list.append(files)

    else:
        continue
print('done - move on to next cell')



dalhousiegazette_volume54_issue13_may_1922__1_.pdf
done - move on to next cell


In [4]:
"""
create function to run in a loop
through the pdf list

"""
def OsCaRizer(pdf_file):

  # Path of the pdf
  #PDF_file = '/content/drive/MyDrive/Colab Notebooks/OCR_Project_Folder/dalhousiegazette_volume54_issue19_november_29_1922.pdf'
  PDF_file = path_of_the_directory + "/" + pdf_file

  #suppress warnings about image size from PIL
  Image.MAX_IMAGE_PIXELS = None   # disables the warning
    
  # Store all the pages of the PDF in a variable
  pages = convert_from_path(PDF_file, 500)
    
  # Counter to store images of each page of PDF to image
  image_counter = 1
    
  # Iterate through all the pages stored above
  for page in pages:
    
      # Declaring filename for each page of PDF as JPG
      # For each page, filename will be:
      # PDF page 1 -> page_1.jpg
      # PDF page 2 -> page_2.jpg
      # PDF page 3 -> page_3.jpg
      # ....
      # PDF page n -> page_n.jpg
      filename = "page_"+str(image_counter)+".jpg"
        
      # Save the image of the page in system
      page.save(filename, 'JPEG')
    
      # Increment the counter to update filename
      image_counter = image_counter + 1
  """
  process the image files to text
  """
  #suppress warnings about image size from PIL
  Image.MAX_IMAGE_PIXELS = None   # disables the warning

  # Variable to get count of total number of pages
  filelimit = image_counter-1
    
  # Creating a text file to write the output
  outfile = ((pdf_file.rsplit( ".", 1)[0]) + '.txt')
    
  # Open the file in append mode so that 
  # All contents of all images are added to the same file
  f = open(outfile, "a")
    
  # Iterate from 1 to total number of pages
  for i in (range(1, filelimit + 1)):
    
      # Set filename to recognize text from
      # These files will be:
      # page_1.jpg
      # page_2.jpg
      # ....
      # page_n.jpg
      filename = "page_"+str(i)+".jpg"
            
      # Recognize the text as string in image using pytesserct
      text = str(((pytesseract.image_to_string(Image.open(filename)))))
    
      # The recognized text is stored in variable text
      # Any string processing may be applied on text
      # Here, basic formatting has been done:
      # In many PDFs, at line ending, if a word can't
      # be written fully, a 'hyphen' is added.
      # The rest of the word is written in the next line.
      # To remove this, we replace every '-\n' to ''.
      text = text.replace('-\n', '')    
    
      # Finally, write the processed text to the file.
      f.write(text)
    
  # Close the file after writing all the text.
  f.close()
  """
  clean out page image files before next instance of loop
  """

  files = os.listdir(path_of_the_directory)
  for f in files:
    if not os.path.isdir(f) and "page" in f:
      os.remove(f)

  print('done - you should see a .txt file named after your .pdf file in your working directory')


In [ ]:
"""
Step 4: Runs the OsCaRizer function.
"""

for k in tqdm(range(len(pdf_list))):
    pdf_file = pdf_list[k]
    OsCaRizer(pdf_file)
    print('the file ' + pdf_file + ' is complete')


  0%|          | 0/1 [00:00<?, ?it/s]

# LDA using Spacey and Gensim 


In [ ]:
#**************************************
# Install libraries and dependencies
#*************************************

# Install libraries and dependencies
!pip install pyLDAvis -qq 
!pip install -qq -U gensim
!pip install spacy -qq
!pip install matplotlib -qq
!pip install seaborn -qq
!python -m spacy download en_core_web_md -qq
!pip install fsspec

import warnings
warnings.filterwarnings('ignore') # this ignores warnings
# Import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
import itertools
import os
from google.colab import files
from tqdm import tqdm

from pathlib import Path
import csv
import itertools

pd.set_option('display.max_columns', None) # this allows you see all columns in pandas

# Our spaCy model:
nlp = en_core_web_md.load() # this will be used to train the algorithm

In [ ]:
"""
Import your data

This should find all .txt files in working dir. 
HOWEVER, it will not find those nested in subdirectories
"""

path_of_the_directory = os.getcwd()
ext = ('.txt')

#define empty df
reports = pd.DataFrame(columns=['file_name','publication_year','content'])
txt_list = []
txt_file_name = []
year_list = []
content_list = []
#get all txt files in the Drive 
for files in os.listdir(path_of_the_directory):

  if files.endswith(ext):
      print(files)
      txt_list.append(files)

      # Creating a text file to write the output
      file_name = ((files.rsplit( ".", 1)[0]))
      txt_file_name.append(file_name)

      #get year
      year_value = ((file_name.rsplit( "_")[-1]))
      year_list.append(year_value)

      #read files into list
      temp_read = Path(files).read_text()
      content_list.append(temp_read)

  else:
      continue
#use to gather lists in a temp df
temp_df = pd.DataFrame.from_records(itertools.zip_longest(txt_file_name, year_list, content_list), columns=['file_name','publication_year','content'])

# take temp_df and concat with perm_df
reports = pd.concat([reports, temp_df], ignore_index=True)

# change this to match your content column name. 
reports['content'] = reports['content'].astype(str)
print()
print('This shows the data types')
print(reports.dtypes)
print()
print('The following is our collected data')
reports.head()

In [ ]:
"""
set number of n-grams here
"""

from nltk import ngrams

def compile_ngrams(text, number_of_n=3, include_unigrams=True):
  ngram_list = []
  # number_of_n controls up to how many n we build an ngram for
  # 2 being bigrams, 3 being trigrams, ect.
  for n in range(number_of_n):
    if n == 0 and not include_unigrams:
      continue
    for ngram in ngrams(text.split(), n + 1):
      ngram_list.append(' '.join(ngram))
  return ngram_list

"""this is a test to make sure the function works"""
print(compile_ngrams("There was a cloud computing conference about big data and natural language processing"))

In [ ]:
"""
data cleaning, setting controlled vocabulary and tokenizing
"""

# Tags I want to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
tokens = []

# Words I really care about that should certainly be in the dictionary
author_assigned_keywords = ["dalhousie university", "dalhousie gazette"] #place anything in here in quotes

for summary in nlp.pipe(reports['content']):
   # build up tokens here
   # using the authors heuristics:
   unigrams = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
   # using ngrams
   # we can use the proj_tok clean text as the "paragraph"
   proj_tok = compile_ngrams(" ".join(unigrams), number_of_n=2, include_unigrams=False)
   # using "author assigned keywords" by checking if they are in the text
   proj_tok += [keyword for keyword in author_assigned_keywords if keyword in summary.text]
   tokens.append(proj_tok)


# Add tokens to new column
reports['tokens'] = tokens 

# Create dictionary
# Apply the Dictionary Object from Gensim, which maps each word to their unique ID:
dictionary = Dictionary(reports['tokens'])
print(dictionary.token2id)

#see the dataframe with a new column called tokens
reports

In [ ]:
"""
set number of topics
"""

# Filter dictionary
"""
the following dictionary filter can result in no terms. adjust to lower values for smaller documents. 
see: https://stackoverflow.com/questions/40840731/valueerror-cannot-compute-lda-over-an-empty-collection-no-terms
"""
#dictionary.filter_extremes(no_below=5, no_above=0.2, keep_n=1000)

# Create corpus
corpus = [dictionary.doc2bow(doc) for doc in reports['tokens']]

# Optimal LDA model building
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=100, num_topics=22, workers = 4, passes=100)

# Print topics
lda_model.print_topics(-1)

# Where does a text belong to
lda_model[corpus][0]
reports['content'][0]

# Visualize topics
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

# Save the report
pyLDAvis.save_html(lda_display, 'index.html')

In [ ]:
"""
Print topics
"""
# Where does a text belong to
temp_clusters = []
for i in range(len(lda_model[corpus])):
  print(lda_model[corpus][i][0])
  temp_clusters.append(lda_model[corpus][i][0])

df2 = pd.DataFrame(temp_clusters, columns=['cluster', 'score'])

#let's see those clusters and the some tokens
#for i in range(0, lda_model.num_topics-1):
for i in range(lda_model.num_topics):
  print ("cluster: " + str(i+1) + "  " + str( lda_model.print_topic(i)))

#create an ordered list and join to reports
reports = pd.concat([reports,df2], axis=1)

#drop the content column as you already have this in the originall l.txt file
reports = reports.drop(columns='content')

#reposition cluster and score
reports = reports[['file_name', 'publication_year', 'cluster', 'score', 'tokens']]

# correct cluster number as ordinal
reports['cluster'] = reports['cluster']+1

#export as excel file
reports.to_csv("LDA_with_clusters.csv", index=False) 

#let's see it!
reports


In [ ]:
"""
coherence tests
"""
# Coherence score using C_umass:
topics = []
score = []
for i in range(1,20,1):
    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
    cm = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
    topics.append(i)
    score.append(cm.get_coherence())
_=plt.plot(topics, score)
_=plt.xlabel('Number of Topics')
_=plt.ylabel('Coherence Score')
plt.show()
